In [5]:
import numpy as np
import matplotlib.pyplot as plt

# ACO Class
class AntColonyOptimization:
    def __init__(self, num_vehicles, vehicle_capacity, customer_data, depot_coords):
        self.num_vehicles = num_vehicles
        self.vehicle_capacity = vehicle_capacity
        self.customer_data = customer_data
        self.depot_coords = depot_coords
        self.distance_matrix = self.calculate_distance_matrix()
        self.pheromone = np.ones_like(self.distance_matrix)  # Initial pheromone level
        self.alpha = 1.0  # Pheromone importance
        self.beta = 2.0  # Distance importance
        self.best_solution = None
        self.best_cost = float('inf')

    def calculate_distance_matrix(self):
        num_customers = len(self.customer_data)
        matrix = np.zeros((num_customers + 1, num_customers + 1))  # Including depot (0)
        for i in range(num_customers + 1):
            for j in range(i + 1, num_customers + 1):
                if i == 0:
                    dist = np.sqrt((self.depot_coords['x'] - self.customer_data[j-1]['x'])**2 +
                                   (self.depot_coords['y'] - self.customer_data[j-1]['y'])**2)
                elif j == 0:
                    dist = np.sqrt((self.depot_coords['x'] - self.customer_data[i-1]['x'])**2 +
                                   (self.depot_coords['y'] - self.customer_data[i-1]['y'])**2)
                else:
                    dist = np.sqrt((self.customer_data[i-1]['x'] - self.customer_data[j-1]['x'])**2 +
                                   (self.customer_data[i-1]['y'] - self.customer_data[j-1]['y'])**2)
                matrix[i, j] = matrix[j, i] = dist
        return matrix

    def choose_next_customer(self, visited, current_customer):
        pheromone = self.pheromone[current_customer]
        pheromone[visited] = 0  # Ignore visited cities

        # Ensure there's no zero distance to avoid NaN
        distance = self.distance_matrix[current_customer] + 1e-6
        probability = pheromone ** self.alpha * (1.0 / distance) ** self.beta

        # Safeguard: If the probability sum is zero, reset it to avoid NaN
        if np.sum(probability) == 0:
            probability = np.ones_like(probability)

        # Normalize the probability to sum to 1
        probability /= np.sum(probability)
        
        # Return the next customer
        return np.random.choice(range(len(self.customer_data) + 1), p=probability)

    def simulate_ant(self):
        visited = [0]  # Start from depot
        total_distance = 0
        current_customer = 0
        while len(visited) < len(self.customer_data) + 1:
            next_customer = self.choose_next_customer(visited, current_customer)
            visited.append(next_customer)
            total_distance += self.distance_matrix[current_customer, next_customer]
            current_customer = next_customer
        return visited, total_distance

    def update_pheromone(self, solutions, decay=0.95):
        self.pheromone *= decay  # Decay pheromone levels
        for route, cost in solutions:
            for i in range(len(route) - 1):
                self.pheromone[route[i], route[i + 1]] += 1.0 / cost

    def run(self, iterations=100):
        for _ in range(iterations):
            ants_solutions = []
            for _ in range(self.num_vehicles):
                visited, total_distance = self.simulate_ant()
                ants_solutions.append((visited, total_distance))
                if total_distance < self.best_cost:
                    self.best_cost = total_distance
                    self.best_solution = visited
            self.update_pheromone(ants_solutions)
        return self.best_solution, self.best_cost

# Split Solution by Trucks
def split_solution_by_trucks(best_solution, num_vehicles):
    routes = []
    customers_per_truck = len(best_solution) // num_vehicles  # Divide customers equally
    
    for i in range(num_vehicles):
        start = i * customers_per_truck
        end = (i + 1) * customers_per_truck if i != num_vehicles - 1 else len(best_solution)
        routes.append(best_solution[start:end])
    
    return routes

# Print and plot truck details
def print_truck_paths(routes, customer_data, depot_coords):
    for i, route in enumerate(routes):
        customer_ids = [customer['id'] for customer in route]
        print(f"Truck {i + 1} path: Depot -> " + " -> ".join(map(str, customer_ids)) + " -> Depot")
        print(f"  Cost (Distance): {sum(customer['demand'] for customer in route)} units")
        print()

def plot_routes(routes, depot_coords, customer_data):
    plt.figure(figsize=(10, 8))
    plt.scatter(depot_coords['x'], depot_coords['y'], color='red', marker='x', label="Depot", s=250)
    for customer in customer_data:
        plt.scatter(customer['x'], customer['y'], color='blue')
        plt.text(customer['x'] + 0.5, customer['y'] + 0.5, f"C{customer['id']}", fontsize=12)

    colors = ['green', 'purple', 'orange', 'pink', 'yellow', 'cyan']
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        route_coords = [(depot_coords['x'], depot_coords['y'])] + [(customer['x'], customer['y']) for customer in route] + [(depot_coords['x'], depot_coords['y'])]
        route_coords = np.array(route_coords)
        plt.plot(route_coords[:, 0], route_coords[:, 1], color=color, marker='o', markersize=5, label=f"Route {i + 1}")
    
    plt.title("Routes des véhicules", fontsize=15)
    plt.xlabel("Coordonnée X")
    plt.ylabel("Coordonnée Y")
    plt.legend()
    plt.grid(True)
    plt.show()

# Main Execution
# Path to the dataset
file_path = '../dataset/c102.txt'  # Adjust the path as necessary

# Step 1: Parse the data from the file
depot_coords, ready_time, due_date, num_vehicles, vehicle_capacity, customer_data = parse_data(file_path)

# Step 2: Initialize and run ACO
aco = AntColonyOptimization(num_vehicles, vehicle_capacity, customer_data, depot_coords)
best_solution, best_cost = aco.run()

# Step 3: Split the solution into individual truck routes
truck_routes = split_solution_by_trucks(best_solution, num_vehicles)

# Step 4: Map the customer data to the routes and plot the routes
routes = []
for truck_route in truck_routes:
    route = [customer_data[i - 1] for i in truck_route]  # Convert customer IDs to data
    routes.append(route)

# Print truck paths and plot the routes
print_truck_paths(routes, customer_data, depot_coords)
plot_routes(routes, depot_coords, customer_data)

ValueError: could not convert string to float: 'C102'

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Parse Data
def parse_data(file_path):
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Recherche de la ligne contenant "VEHICLE"
        vehicle_index = next(i for i, line in enumerate(lines) if "VEHICLE" in line)
        max_vehicles = int(lines[vehicle_index + 2].split()[0])  # Nombre de véhicules
        vehicle_capacity = int(lines[vehicle_index + 2].split()[1])  # Capacité des véhicules

        # Recherche de la ligne contenant "CUSTOMER"
        customer_index = next(i for i, line in enumerate(lines) if "CUSTOMER" in line)
        
        # Initialisation des variables pour le dépôt
        depot_coords = None
        ready_time = None
        due_date = None
        customer_data = []

        # Parcours des lignes des clients
        for line in lines[customer_index + 2:]:
            data = line.strip().split()
            
            # Si la ligne a les bonnes données (7 éléments), traiter les informations
            if len(data) == 7:
                customer_id = int(data[0])
                xcoord = float(data[1])
                ycoord = float(data[2])
                demand = int(data[3])
                ready_time_cust = int(data[4])
                due_date_cust = int(data[5])
                service_time = int(data[6])
                
                # Si c'est la ligne du dépôt (id == 0), on prend ces valeurs
                if customer_id == 0:
                    depot_coords = {'x': xcoord, 'y': ycoord}
                    ready_time = ready_time_cust
                    due_date = due_date_cust
                    print(f"Dépôt trouvé: {depot_coords}, Ready time: {ready_time}, Due date: {due_date}")
                else:
                    # Sinon, c'est un client classique
                    customer_data.append({
                        'id': customer_id,
                        'x': xcoord,
                        'y': ycoord,
                        'demand': demand,
                        'ready_time': ready_time_cust,
                        'due_date': due_date_cust,
                        'service_time': service_time
                    })

        # Vérification si le dépôt a été trouvé
        if depot_coords is None:
            raise ValueError("Coordonnées du dépôt manquantes ou incorrectes dans le fichier.")
        
        # Retourner les informations : coordonnées du dépôt, horaires, et les données des clients
        return depot_coords, ready_time, due_date, max_vehicles, vehicle_capacity, customer_data
    except Exception as e:
        print(f"An error occurred while parsing data: {e}")
        return None


# Test parsing function with a sample file
file_path = '../dataset/c101.txt'  # Or your relevant file
result = parse_data(file_path)
if result:
    print("Parsing successful.")
else:
    print("Parsing failed.")




# ACO Class
class AntColonyOptimization:
    def __init__(self, num_vehicles, vehicle_capacity, customer_data, depot_coords):
        self.num_vehicles = num_vehicles
        self.vehicle_capacity = vehicle_capacity
        self.customer_data = customer_data
        self.depot_coords = depot_coords
        self.distance_matrix = self.calculate_distance_matrix()
        self.pheromone = np.ones_like(self.distance_matrix)  # Initial pheromone level
        self.alpha = 1.0  # Pheromone importance
        self.beta = 2.0  # Distance importance
        self.best_solution = None
        self.best_cost = float('inf')
        self.best_routes = []

    def calculate_distance_matrix(self):
        num_customers = len(self.customer_data)
        matrix = np.zeros((num_customers + 1, num_customers + 1))  # Including depot (0)
        for i in range(num_customers + 1):
            for j in range(i + 1, num_customers + 1):
                if i == 0:
                    dist = np.sqrt((self.depot_coords['x'] - self.customer_data[j-1]['x'])**2 +
                                   (self.depot_coords['y'] - self.customer_data[j-1]['y'])**2)
                elif j == 0:
                    dist = np.sqrt((self.depot_coords['x'] - self.customer_data[i-1]['x'])**2 +
                                   (self.depot_coords['y'] - self.customer_data[i-1]['y'])**2)
                else:
                    dist = np.sqrt((self.customer_data[i-1]['x'] - self.customer_data[j-1]['x'])**2 +
                                   (self.customer_data[i-1]['y'] - self.customer_data[j-1]['y'])**2)
                matrix[i, j] = matrix[j, i] = dist
        return matrix

    def choose_next_customer(self, visited, current_customer, truck_load):
        pheromone = self.pheromone[current_customer]
        pheromone[visited] = 0  # Ignore visited cities

        # Ensure there's no zero distance to avoid NaN
        distance = self.distance_matrix[current_customer] + 1e-6
        probability = pheromone ** self.alpha * (1.0 / distance) ** self.beta

        # Safeguard: If the probability sum is zero, reset it to avoid NaN
        if np.sum(probability) == 0:
            probability = np.ones_like(probability)

        # Normalize the probability to sum to 1
        probability /= np.sum(probability)

        # Choose the next customer based on the probabilities
        next_customer = np.random.choice(range(len(self.customer_data) + 1), p=probability)

        # Check if adding the next customer exceeds the truck's capacity
        if next_customer > 0:  # Skip depot (0)
            if truck_load + self.customer_data[next_customer-1]['demand'] <= self.vehicle_capacity:
                return next_customer
            else:
                # If the truck would be overloaded, try again
                return self.choose_next_customer(visited, current_customer, truck_load)
        else:
            return next_customer

    def simulate_ant(self):
        visited = [0]  # Start from depot
        truck_load = 0
        total_distance = 0
        current_customer = 0
        route = []

        while len(visited) < len(self.customer_data) + 1:
            next_customer = self.choose_next_customer(visited, current_customer, truck_load)
            visited.append(next_customer)
            route.append(next_customer)

            # Update load and distance
            if next_customer > 0:  # Skip depot (0)
                truck_load += self.customer_data[next_customer-1]['demand']
            total_distance += self.distance_matrix[current_customer, next_customer]
            current_customer = next_customer
        return route, total_distance

    def update_pheromone(self, solutions, decay=0.95):
        self.pheromone *= decay  # Decay pheromone levels
        for route, cost in solutions:
            for i in range(len(route) - 1):
                self.pheromone[route[i], route[i + 1]] += 1.0 / cost

    def run(self, iterations=100):
        for _ in range(iterations):
            ants_solutions = []
            for _ in range(self.num_vehicles):
                route, total_distance = self.simulate_ant()
                ants_solutions.append((route, total_distance))
                if total_distance < self.best_cost:
                    self.best_cost = total_distance
                    self.best_solution = route
            self.update_pheromone(ants_solutions)
        return self.best_solution, self.best_cost


# Split Solution by Trucks
def split_solution_by_trucks(best_solution, vehicle_capacity, customer_data):
    routes = []
    current_route = []
    current_load = 0
    for customer_id in best_solution:
        if customer_id > 0:  # Skip depot (0)
            customer = customer_data[customer_id - 1]
            if current_load + customer['demand'] <= vehicle_capacity:
                current_route.append(customer)
                current_load += customer['demand']
            else:
                routes.append(current_route)
                current_route = [customer]
                current_load = customer['demand']
    if current_route:
        routes.append(current_route)
    return routes


# Print and plot truck details
def print_truck_paths(routes):
    for i, route in enumerate(routes):
        customer_ids = [customer['id'] for customer in route]
        print(f"Truck {i + 1} path: Depot -> " + " -> ".join(map(str, customer_ids)) + " -> Depot")
        print(f"  Total load: {sum(customer['demand'] for customer in route)} units")
        print(f"  Cost (Distance): {sum(customer['demand'] for customer in route)} units")
        print()


def plot_routes(routes, depot_coords, customer_data):
    plt.figure(figsize=(10, 8))
    plt.scatter(depot_coords['x'], depot_coords['y'], color='red', marker='x', label="Depot", s=250)
    for customer in customer_data:
        plt.scatter(customer['x'], customer['y'], color='blue')
        plt.text(customer['x'] + 0.5, customer['y'] + 0.5, f"C{customer['id']}", fontsize=12)

    colors = ['green', 'purple', 'orange', 'pink', 'yellow', 'cyan']
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        route_coords = [(depot_coords['x'], depot_coords['y'])] + [(customer['x'], customer['y']) for customer in route] + [(depot_coords['x'], depot_coords['y'])]
        route_x, route_y = zip(*route_coords)
        plt.plot(route_x, route_y, marker='o', color=color, label=f"Truck {i + 1}")

    plt.legend()
    plt.title("Truck Routes")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.grid(True)
    plt.show()


# Main Code to Run
file_path = '../Dataset/c101.txt'  # Or '/mnt/data/c102.txt'

# Step 1: Parse the data from the file
depot_coords, ready_time, due_date, num_vehicles, vehicle_capacity, customer_data = parse_data(file_path)

# Step 2: Initialize and run ACO
aco = AntColonyOptimization(num_vehicles, vehicle_capacity, customer_data, depot_coords)
best_solution, best_cost = aco.run()

# Step 3: Split the solution into individual truck routes
truck_routes = split_solution_by_trucks(best_solution, vehicle_capacity, customer_data)

# Step 4: Map the customer data to the routes and plot the routes
routes = []
for truck_route in truck_routes:
    route = truck_route  # Keep customer data
    routes.append(route)

# Print truck paths and plot the routes
print_truck_paths(routes)
plot_routes(routes, depot_coords, customer_data)

Dépôt trouvé: {'x': 40.0, 'y': 50.0}, Ready time: 0, Due date: 1236
Parsing successful.
Dépôt trouvé: {'x': 40.0, 'y': 50.0}, Ready time: 0, Due date: 1236


: 